In [1]:
from jupyter_plotly_dash import JupyterDash

import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State


import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME ##### fixed
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name fixed
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object fixed
df = pd.DataFrame.from_records(shelter.find({}))


#FIX ME Add in Grazioso Salvare’s logo fixed
image_filename = "GraziosoSalvareLogo.png" # replace with your own image fixed
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design fixed
#FIX ME Also remember to include a unique identifier such as your name or date added
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    
    html.Br(),
     html.Hr(),
    html.Header('Tyler Owens CS340'),
    html.Hr(),
    
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image', style={'height':'10%', 'width':'10%'}),
    html.Br(),
    html.B("Selectable Rescue Types:"),
    html.Br(),
    dcc.RadioItems(
        id='radio-id',
        options=[
            {"label": "Water Rescue", "value": "wr"},
            {"label": "Mountain Rescue", "value": "mr"},
            {"label": "Disaster Rescue", "value": "dr"},
            {"label": "Reset", "value": "r"},

        ],
        #value='MTL',
        labelStyle={'display': 'inline-block'}
    ),
    html.Br(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        editable=False,
        filter_action="native",
        row_selectable="single",

        selected_columns=[],
        
        page_action="native",


        page_current=0,
        page_size=10


    ),
    html.Br(),
    html.Hr(),
    html.Br(),
    html.Div(className='row',
            style={'display': 'flex', "width": "500"},
            children=[
                html.Div(
                    className='row',
                    style={'display': 'flex'},
                    children=[
                        # where the pie chart will go
                        dcc.Graph(
                            id='graph-id',
                        ),
                        # where the map will go
                        html.Div(
                            id='map-id',
                            className='col s12 m6',
                        ),
                    ]
                ),
            ])
    
    


])
'''
    html.Div(
        dbc.Row(
            [
                dbc.Col(html.Div(id="graph_id"), width=6),
                dbc.Col(html.Div(
                    id='map-id',
                    className='col s12 m6',
                ), width=6),
            ],
        ),
    ),
    '''

@app.callback(Output("datatable-id", "data"), [Input("radio-id", "value")])
def update_datatable(value):
    if value == "wr":
        df = pd.DataFrame.from_records(shelter.find_water_rescue())
    elif value == "mr":
        df = pd.DataFrame.from_records(shelter.find_mountain_rescue())
    elif value == "dr":
        df = pd.DataFrame.from_records(shelter.find_disaster_rescue())
    elif value == "r":
        df = pd.DataFrame.from_records(shelter.find({}))
    else:
        df = pd.DataFrame.from_records(shelter.find({})) 
    return df.to_dict("records")
'''
@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "derived_virtual_data"))
def update_graph(viewData):
    raise PreventUpdate
    return []
'''

@app.callback(
   Output('graph-id', "figure"),[
   Input('datatable-id', "derived_virtual_data")])
def update_graph(allData):
    dff = pd.DataFrame(allData)
    piechart = px.pie(
        data_frame=dff,
        # pie chart measures outcome_type column
        names=dff['outcome_type'],
        hole=.3,
    )
    return piechart



@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, viewRow):
    dff = df
    #stops the map from updating if it doesnt have a target
    if viewRow is None or len(viewRow) == 0:
        raise PreventUpdate

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, 
               center=[#30.75,-97.48
                    float(dff.iloc[viewRow, 13].values[0]),
                    float(dff.iloc[viewRow, 14].values[0]),

               ], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[
                    float(dff.iloc[viewRow, 13].values[0]),
                    float(dff.iloc[viewRow, 14].values[0]),], 
                      children=[
                dl.Tooltip(dff.iloc[viewRow,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[viewRow,9])
                ])
            ])
        ])
    ]







app